In [ ]:
import time
from fmu.sumo.explorer import Explorer
from fmu.sumo.explorer.objects.table import AggregatedTable
import seaborn as sns
%matplotlib inline

### Initialize Explorer

In [ ]:
sumo = Explorer("dev")

### Get Case

In [ ]:
# Get case by name (name is not guaranteed to be unique)
case = sumo.cases.filter(name="drogon_ahm-2023-02-22")[0]


### Finding info about tables connected to case

In [ ]:
tables = case.tables
print(f"Table names: {tables.names}")
print(f"Table tags: {tables.tagnames}")

### Finding info related to specific table name

In [ ]:
tables = tables.filter(name="summary")
print(f"Tagname check: {tables.tagnames}")

### The filter way

In [ ]:
table = case.tables.filter("summary", "eclipse", "iter-0", aggregation="collection", column="FOPT")
print(len(table))
print(table.tagnames)
print(table.names)
table = table[0]

table.dataframe

### For user friendly access to summary data


In [ ]:
# Get case surfaces
summary = AggregatedTable(case, "summary", "eclipse", "iter-0", aggregation="collection")

for key in summary["FOPR"]["fmu"]["iteration"]["parameters"]["GLOBVAR"].keys():
    print(key)

### If you prefer pandas to arrow

In [ ]:
summary = AggregatedTable(case, "summary", "eclipse", "iter-0")

table = summary["FOPT"].dataframe

### Inplace volumes

In [ ]:
# Get case surfaces
inplace = AggregatedTable(case, "geogrid", "vol", "iter-0")

inplace["STOIIP_OIL"].dataframe.groupby(["ZONE", "REAL"])["STOIIP_OIL"].agg("sum")["Therys"].plot(kind="hist")

### Summary speedtest

In [ ]:
start = time.perf_counter()
count = 0
for col_name in summary.columns[:20]:
    vector = summary[col_name]
    print(vector.dataframe.head())
    count += 1
print(f"{count} cols in total time: {time.perf_counter() - start: .1f} s")

### Inplace speedtest

In [ ]:
start = time.perf_counter()
count = 0
for col_name in inplace.columns[:20]:
    col = inplace[col_name]
    print(col)
    count += 1
print(f"{count} cols in total time: {time.perf_counter() - start: .1f} s")